In [ ]:
!pip install streamlit
!pip install streamlit-folium
!pip install geopandas

In [ ]:
import os

os.chdir('/content/drive/MyDrive/Omdena/Osun Nigeria/OOP_App')

!pwd

/content/drive/MyDrive/Omdena/Osun Nigeria/OOP_App


In [ ]:
%%writefile app.py

import streamlit as st
from apps import home, crop_disease_detector, crop_detector, weather_predictor
from PIL import Image

with st.container():
    proj_title_col, logo_col = st.columns([6,1])

    with proj_title_col:
      st.header('Improving Food Security and Crop Yield in Nigeria using Machine Learning')

    with logo_col:
      logo = Image.open('/content/drive/MyDrive/Omdena/Osun Nigeria/App/osun_chapter.png')
      logo = logo.resize((75,75))
      st.image(logo)

PAGES = {
    "Home": home,
    "Crop Disease Detection": crop_disease_detector,
    'Crop Classification': crop_detector,
    'Weather Forecast': weather_predictor
}

selection = st.sidebar.radio("Menu", list(PAGES.keys()))
page = PAGES[selection]
page.app()

Overwriting app.py


In [ ]:
%%writefile apps/home.py

import streamlit as st

def app():
  st.subheader('Problem Statement')
  st.write("According to the Food and Agriculture Organisation of the United Nations, 2018, \
  approximately 88 % of the farmers in Nigeria engage in \
  agricultural production at a subsistence level, and they lack sustainable \
  farming knowledge and practices. Also, Nigeria is endowed with \
  different climatic conditions and soil quality which leads to lackluster \
  crop production. ")

  st.markdown('##')

  st.subheader('Objectives of the Project')
  st.write('This project is aimed at helping farmers to boost their \
  farm produce and plan their farming system.')

Overwriting apps/home.py


In [ ]:
%%writefile apps/app_detector.py

import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array
from PIL import Image
import cv2
import utils

class AppDetector:

  def __init__(self, data, target_size):
    self.data = data
    self.target_size = target_size
  

  def app(self):
    #new_image = ''
    has_pred = False
    has_replaced_image = False
    #pred = ''

    model = utils.load_model(self.data)

    
    with st.container():
        upload_col, predict_col = st.columns([6,1])
        with upload_col:
          
          #Upload the image
          uploaded_file = st.file_uploader('Choose a file',type=["jpg", 'png'])
          if uploaded_file is not None:
            img = Image.open(uploaded_file)
            img_display = img.resize((700, 500))
            st.image(img_display)

            #Preprocess image
            if self.data == 'Crops':
              img = utils.crop_preprocess_img(img, self.target_size)
            else:
              img = utils.disease_preprocess_img(img, self.target_size)
            
            new_image = img
            has_replaced_image = True
            
            
        with predict_col:
          #Predict the disease label for the image
            if st.button('Predict'):
              if has_replaced_image:
                predictions = utils.predict(new_image, model)
                pred = np.argmax(predictions)
                has_pred = True
              else:
                st.error('Please upload an image')

    #This container is for the description of the predicted disease of the image            
    with st.container():
      if has_pred:
        class_ = utils.get_class(self.data, pred)
        st.title(class_)
        if self.data != 'Crops':
          with st.expander("Read disease description"):
            #disease = utils.get_class(self.data, pred)
            #st.title(disease)
            disease_desc = utils.get_description(class_)
            st.markdown(disease_desc, unsafe_allow_html=True)

Overwriting apps/app_detector.py


In [ ]:
%%writefile apps/crop_disease_detector.py

import streamlit as st
from apps.app_detector import AppDetector

TARGET_SIZE = 100

def app(): 
  selection = st.selectbox('Crop Dataset', ('Maize', 'Rice'))
  app_detect = AppDetector(selection, TARGET_SIZE)
  app_detect.app()

Overwriting apps/crop_disease_detector.py


In [ ]:
%%writefile apps/crop_detector.py

import streamlit as st
from apps.app_detector import AppDetector

TARGET_SIZE = 224

def app():
  app_detect = AppDetector('Crops', TARGET_SIZE)
  app_detect.app()

Overwriting apps/crop_detector.py


In [ ]:
%%writefile apps/weather_predictor.py
#version 4

import streamlit as st
import pandas as pd
import folium
from folium.plugins import BeautifyIcon
from streamlit_folium import folium_static
import geopandas as gpd
import shapely
import calendar
import branca.colormap as cm

@st.cache(persist=True, allow_output_mutation=True)
def process_data():
  all_three = pd.read_csv('/content/drive/MyDrive/Omdena/Osun Nigeria/OOP_App/csv/temp_pr_spei.csv')
  all_three = (gpd.GeoDataFrame(all_three, crs='epsg:4326', geometry= all_three.apply(lambda x: 
                  shapely.geometry.box(x['lon']-0.25, x['lat']-0.25, x['lon']+0.25, x['lat']+0.25 ), axis=1)))
  
  spei = pd.read_csv('/content/drive/MyDrive/Omdena/Osun Nigeria/OOP_App/csv/historical_spei_gamma.csv')
  spei_grid = gpd.GeoDataFrame(spei,crs='epsg:4326', geometry=spei.apply(lambda x: shapely.geometry.box(x['lon']-0.25, x['lat']-0.25, x['lon']+0.25, x['lat']+0.25 ), axis=1))
  
  temp = pd.read_csv('/content/drive/MyDrive/Omdena/Osun Nigeria/OOP_App/csv/historical_tmp.csv')
  temp_grid = gpd.GeoDataFrame(temp,crs='epsg:4326', geometry=temp.apply(lambda x: shapely.geometry.box(x['lon']-0.25, x['lat']-0.25, x['lon']+0.25, x['lat']+0.25 ), axis=1))

  rain = pd.read_csv('/content/drive/MyDrive/Omdena/Osun Nigeria/OOP_App/csv/historical_pr.csv')
  rain_grid = gpd.GeoDataFrame(rain,crs='epsg:4326', geometry=rain.apply(lambda x: shapely.geometry.box(x['lon']-0.25, x['lat']-0.25, x['lon']+0.25, x['lat']+0.25 ), axis=1))

  return all_three, temp_grid, rain_grid, spei_grid

def app():
  all_three, temp, rain, spei = process_data()

  gdf = all_three.copy()
  temp = temp.copy()
  rain = rain.copy()
  spei = spei.copy()

  x = all_three['lon'].median()
  y = all_three['lat'].median()


  option_dict = {'Temperature':'tmp', 'Rainfall':'pre', 'SPEI':'spei_gamma_03'}

  # st.markdown('## Weather and Drought Analysis')

  st.write('''## Weather and Drought Analysis


  The map below shows the spatial and temporal distribution of Temperature and Rainfall, and also one drought index, 
  the Standardized Precipitation Evapotranspiration Index (SPEI).


  The SPEI is a multiscalar drought index based on climatic data. It can be used for determining the onset, 
  duration and magnitude of drought conditions with respect to normal conditions in a variety of natural and managed 
  systems such as crops, ecosystems, rivers, water resources, etc. SPEI is a standardized index. Values is between -1 and -2
  are said to mean moderate drought, between -2 and -3 are said to mean severe drought, whereas values lower than -3 are 
  said to mean extreme drought.


  The figures show that the Northern regions are hotter and drier, while the Southern regions are cooler and wetter.
  ''')


  selected = st.radio('Pick a variable', options=option_dict.keys())

  v = option_dict[selected]

  if selected == 'Temperature':
    color = 'OrRd'
  elif selected =='Rainfall':
    color = 'Blues'
  elif selected == 'SPEI':
    color = 'RdYlBu'

  mon_annual = st.radio('Select level of aggregation', options = ['Annual', 'Monthly'])

  if mon_annual == 'Annual':
    if selected in ['Temperature', 'SPEI']:
      agg_func = 'mean'
    elif selected == 'Rainfall':
      agg_func = 'sum'
    
    temp2 = gdf.groupby(['lat', 'lon'])[[v]].agg(agg_func).merge(gdf[['geometry', 'lat', 'lon']].drop_duplicates(), on=['lat','lon']).reset_index()
    temp2.columns = ['division', 'lat', 'lon', v, 'geometry']
    
    temp2.set_crs(epsg=4326, inplace=True)


    m = folium.Map(location=[y,x], zoom_start=6)


    choropleth = folium.Choropleth(
        geo_data=temp2,
        name="choropleth",
        data=temp2,
        columns=['division',v],
        key_on="feature.properties.division",
        fill_color=color,
        fill_opacity=0.7,
        line_opacity=0.1,
        legend_name=v,
    ).add_to(m)

    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip([v])
    )

    st.write('Annual distribution of '+ selected)
    folium_static(m)

  elif mon_annual=='Monthly':

      month = st.selectbox('Pick a month', calendar.month_name[1:])
      
      m = folium.Map(location=[y,x], zoom_start=6)


      choropleth = folium.Choropleth(
          geo_data=gdf.loc[gdf['Month_names']==month].reset_index()[['division', v,'geometry']],
          name="choropleth",
          data=gdf.loc[gdf['Month_names']==month].reset_index()[['division', v,'geometry']],
          columns=['division',v],
          key_on="feature.properties.division",
          fill_color=color,
          fill_opacity=0.7,
          line_opacity=0.1,
          legend_name=v,
      ).add_to(m)

      choropleth.geojson.add_child(
          folium.features.GeoJsonTooltip([v])
      )


      st.write(selected + ' for the month of '+ month)
      folium_static(m)
      


  st.header('Analyzing trends in Temperature, rainfall, and SPEI')

  st.write('''The Modified Mann Kendall test was used to assess the trend in rainfall, temperature, and SPEI for different months.
  The data was prepared for the test by smoothing it with a 50 year moving average and a 10 year time step.
  The grid points are colored by the slope, in case of temperature it is change in degree Celsius per decade, for rainfall it is 
  change in mm per decade, and for spei it is change in SPEI per decade. Grid points that have a significant increase 
  are marked with a "+" sign, and those that have a significant decrease are marked with a "-" sign.


  The figures show that temperature is increasing in most locations, and rainfall is decreasing. The SPEI is also decreasing
  in many locations which means that chances of drought are increasing.''')
  st.subheader('Trends in Temperature')

  month1 = st.selectbox('Pick a month', calendar.month_name[1:], key='month1-key')
  st.write('Trends in Temperature for the month of ' + month1 + ' in degrees Celsius/decade')

  m1 = folium.Map(location=[y,x], zoom_start=6)

  choropleth = folium.Choropleth(
      geo_data=temp.loc[temp['month']==month1, ['division', 'slope', 'geometry']],
      name="choropleth",
      data=temp.loc[temp['month']==month1,['division', 'slope', 'geometry']],
      columns=['division','slope'],
      key_on="feature.properties.division",
      fill_color='PuOr',
      fill_opacity=0.7,
      line_opacity=0.1,
      legend_name='slope',
  ).add_to(m1)


  choropleth.geojson.add_child(
      folium.features.GeoJsonTooltip(['slope'])
  )

  for index, row in temp.loc[(temp['month']==month1 )& temp['h'] & (temp['z'] >0)].iterrows():
    icon_plus = BeautifyIcon(
      icon='plus',
      icon_shape=None, 
      border_color=None, 
      border_width=0,
      background_color='')
    folium.Marker(
          location=[row['lat'], row['lon']], 
          icon=icon_plus).add_to(m1)

  for index, row in temp.loc[(temp['month']== month1 ) & temp['h'] & (temp['z'] <0)].iterrows():
    icon_minus = BeautifyIcon(
      icon='minus',
      icon_shape=None, 
      border_color=None, 
      border_width=0,
      background_color='')
    folium.Marker(
          location=[row['lat'], row['lon']], 
          icon=icon_minus).add_to(m1)

  folium_static(m1)

  st.subheader('Trends in Rainfall')
  st.write('Trends in Rainfall for the month of ' + month1 + ' in mm/decade')

  m2 = folium.Map(location=[y,x], zoom_start=6)

  choropleth = folium.Choropleth(
      geo_data=temp.loc[rain['month']==month1, ['division', 'slope', 'geometry']],
      name="choropleth",
      data=temp.loc[rain['month']==month1,['division', 'slope', 'geometry']],
      columns=['division','slope'],
      key_on="feature.properties.division",
      fill_color='PuOr',
      fill_opacity=0.7,
      line_opacity=0.1,
      legend_name='slope',
  ).add_to(m2)


  choropleth.geojson.add_child(
      folium.features.GeoJsonTooltip(['slope'])
  )

  for index, row in rain.loc[(rain['month']==month1 )& temp['h'] & (temp['z'] >0)].iterrows():
    icon_plus = BeautifyIcon(
      icon='plus',
      icon_shape=None, 
      border_color=None, 
      border_width=0,
      background_color='')
    folium.Marker(
          location=[row['lat'], row['lon']], 
          icon=icon_plus).add_to(m2)

  for index, row in rain.loc[(rain['month']==month1 ) & rain['h'] & (rain['z'] <0)].iterrows():
    icon_minus = BeautifyIcon(
      icon='minus',
      icon_shape=None, 
      border_color=None, 
      border_width=0,
      background_color='')
    folium.Marker(
          location=[row['lat'], row['lon']], 
          icon=icon_minus).add_to(m2)

  folium_static(m2)


  st.subheader('Trends in SPEI')
  st.write('Trends in SPEI/decade for the month of ' + month1)

  m3 = folium.Map(location=[y,x], zoom_start=6)

  choropleth = folium.Choropleth(
      geo_data=spei.loc[spei['month']==month1, ['division', 'slope', 'geometry']],
      name="choropleth",
      data=spei.loc[spei['month']==month1,['division', 'slope', 'geometry']],
      columns=['division','slope'],
      key_on="feature.properties.division",
      fill_color='PuOr',
      fill_opacity=0.7,
      line_opacity=0.1,
      legend_name='slope',
  ).add_to(m3)


  choropleth.geojson.add_child(
      folium.features.GeoJsonTooltip(['slope'])
  )

  for index, row in spei.loc[(spei['month']==month1 )& spei['h'] & (spei['z'] >0)].iterrows():
    icon_plus = BeautifyIcon(
      icon='plus',
      icon_shape=None, 
      border_color=None, 
      border_width=0,
      background_color='')
    folium.Marker(
          location=[row['lat'], row['lon']], 
          icon=icon_plus).add_to(m3)

  for index, row in spei.loc[(spei['month']==month1 ) & spei['h'] & (spei['z'] <0)].iterrows():
    icon_minus = BeautifyIcon(
      icon='minus',
      icon_shape=None, 
      border_color=None, 
      border_width=0,
      background_color='')
    folium.Marker(
          location=[row['lat'], row['lon']], 
          icon=icon_minus).add_to(m3)

  folium_static(m3)

Overwriting apps/weather_predictor.py


In [ ]:
%%writefile utils.py

import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array
from PIL import Image
import cv2

@st.cache(allow_output_mutation=True)
def load_model(dataset):
  """
  Loads the models.
  """

  if dataset == 'Maize':
    model = tf.keras.models.load_model('/content/drive/MyDrive/Omdena/Osun Nigeria/Model/model_maize_final.h5')
  elif dataset == 'Rice':
    model = tf.keras.models.load_model('/content/drive/MyDrive/Omdena/Osun Nigeria/Model/model_rice.h5')
  elif dataset == 'Crops':
    model = tf.keras.models.load_model('/content/drive/MyDrive/crop_classification.h5')
  return model

def crop_preprocess_img(image, TARGET_SIZE):
  pil_img_rgb = image.convert('RGB') 
  cv2_img = np.array(pil_img_rgb) #convert PIL Image to cv2 format
  #img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #convert BGR to RGB
  img = cv2.resize(cv2_img, (TARGET_SIZE, TARGET_SIZE)) #resize the image to (224, 224)
  img = img_to_array(img) #convert to numpy array
  img = img/255 #normalize the image
  img = np.expand_dims(img, axis = 0) #expands dimension to one sample
  return img

def disease_preprocess_img(image, TARGET_SIZE):
  """ Preprocess the image in the same way the images for model building were processed. 
  
  image: uploaded image

  Returns: image preprocessed using resnet50's preprocess_input
  """

  img = image.resize((TARGET_SIZE, TARGET_SIZE))
  img = img_to_array(img) #convert to numpy array
  img = np.expand_dims(img, axis = 0) #expands dimension to one sample
  img = preprocess_input(img) #preprocess function by resnet50
  return img

def predict(image, model):
  """ Predicts the disease of the image.

  image: preprocessed image
  model: model for maize or rice leaf disease datasets

  Returns: probability for each class/disease
  """
  predictions = model.predict(image)
  return predictions

def get_class(crop, idx):
  """ Gets the disease label for the predicted image.

  crop: crop type - maize or rice
  idx: column index of the class with the highest probability in the predictions

  Returns: disease label

  """
  if crop == 'Maize':
    CLASSES = {
        0: 'Blight',
        1: 'Common Rust',
        2: 'Gray Leaf Spot',
        3: 'Healthy'
    }
  elif crop == 'Rice':
    CLASSES = {
        0: 'Bacterial Leaf Blight',
        1: 'Brown Spot',
        2: 'Leaf Smut'
    }
  elif crop == 'Crops':
    CLASSES = {
        0: 'Rice',
        1: 'Maize'
    }

  return CLASSES[idx]

def get_description(disease):
  """ Gets the disease description

    disease: predicted disease for the image
  """
  maize_blight = f"""
  <p> 
  The tan lesions of northern corn leaf blight are slender and oblong tapering at the ends ranging in size between 1 to 6 inches. </p>
  <p>Lesions run parallel to the leaf margins beginning on the lower leaves and moving up the plant. They may coalesce and cover the enter leaf.
  </p>
  <p>Spores are produced on the underside of the leaf below the lesions giving the appearance of a dusty green fuzz.</p>

  <p> <i>Source: https://cals.cornell.edu/field-crops/corn/diseases-corn/</i></p>
  """

  maize_common_rust = f"""
  <p> Common rust is caused by the fungus Puccinia sorghi. </p>
  <p>Small, round to elongate brown pustules form on both leaf surfaces and other above ground parts of the plant. </p>
  <p>As the pustules mature they become brown to black. If disease is severe, the leaves may yellow and die early
  </p>

  <p> <i>Source: https://cals.cornell.edu/field-crops/corn/diseases-corn/</i></p>
  """
  maize_gray_leaf_spot = f"""
  <p> Gray leaf spot is caused by the fungus Cercospora zeae-maydis. 
  Lesions start as a small dot surrounded by yellow halo, and then will elongate over time parallel to the veins becoming pale brown to gray.
  </p>

  <p> <i>Source: https://cals.cornell.edu/field-crops/corn/diseases-corn/</i></p>
  """

  maize_healthy = f""""""

  rice_bacterial_leaf_blight = f"""
  <p> Rice Bacterial Blight is a deadly bacterial disease that is among the most destructive afflictions of cultivated rice (Oryza sativa and O. glaberrima). 
  In severe epidemics, crop loss may be as high as 75 percent, and millions of hectares of rice are infected annually.
  </p>
  """

  rice_brown_spot = f"""
  <p> Brown spot is caused by the fungus Cochliobolus miyabeanus. Also called Helminthosporium leaf spot, it is one of the most prevalent rice diseases. 
  It is a fungal disease that infects the coleoptile, leaves, leaf sheath, panicle branches, glumes, and spikelets.
  </p>
  """

  rice_leaf_smut = f"""
  <p> Leaf smut, caused by the fungus Entyloma oryzae, is a widely distributed, but somewhat minor, disease of rice. 
  The fungus produces slightly raised, angular, black spots (sori) on both sides of the leaves.
  </p>
  """

  dict_description = {
      'Blight': maize_blight,
      'Common Rust': maize_common_rust,
      'Gray Leaf Spot': maize_gray_leaf_spot,
      'Healthy': maize_healthy,
      'Bacterial Leaf Blight': rice_bacterial_leaf_blight,
      'Brown Spot': rice_brown_spot,
      'Leaf Smut': rice_leaf_smut
  }

  return dict_description[disease]


Overwriting utils.py


In [ ]:
 !streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 2.998s
your url is: https://kind-cheetah-83.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.196.100.208:8501

2021-10-30 14:44:59.419383: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-30 14:44:59.419469: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (85423515b07e): /proc/driver/nvidia/version does not exist
2021-10-30 14:48:20.860 NumExpr defaulting to 2 threads.
2021-10-30 14:56:13.418353: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
